# Model Interation 3 (final)
Esta es la mejor iteracion del modelo.

In [2]:
import pandas as pd
import numpy as np
import gdown
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.calibration import calibration_curve
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
# Cargar el archivo CSV desde local
df = pd.read_csv('../Modelo/gamesWcategoricas.csv')


/var/folders/yn/vbht61m97rg6wrrbgs95_9k00000gn/T/ipykernel_89962/477908195.py:2: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../Modelo/gamesWcategoricas.csv')  # ajustá el path si hace falta


In [5]:
# Revisar columnas clave para ingeniería
print("Valores únicos de 'Release date':")
print(df['Release date'].unique()[:10])  # ver algunos

print("\nTipo de 'Release date':", df['Release date'].dtype)

print("\nValores únicos de 'Price':")
print(df['Price'].unique()[:10])  # ver algunos

print("\nTipo de 'Price':", df['Price'].dtype)


Valores únicos de 'Release date':
[2008 2017 2021 2020 2022 2014 2019 2016 2018 2010]

Tipo de 'Release date': int64

Valores únicos de 'Price':
[19.99  0.99  4.99  5.99  0.   10.99  9.99 14.99  3.99 23.99]

Tipo de 'Price': float64


In [6]:
# Feature: release_year
df['release_year'] = df['Release date']

# Feature: price_range
def clasificar_precio(precio):
    if precio == 0:
        return 'free'
    elif precio < 10:
        return 'low'
    elif precio < 30:
        return 'medium'
    else:
        return 'high'

df['price_range'] = df['Price'].apply(clasificar_precio)

# Verificamos
print(df[['release_year', 'Price', 'price_range']].head())
print("\nDistribución de price_range:")
print(df['price_range'].value_counts())


   release_year  Price price_range
0          2008  19.99      medium
1          2017   0.99         low
2          2021   4.99         low
3          2020   5.99         low
4          2020   0.00        free

Distribución de price_range:
price_range
low       66873
free      23247
medium    19066
high       2266
Name: count, dtype: int64


In [7]:
# Aplicar One-Hot Encoding a price_range
df_encoded = pd.get_dummies(df, columns=['price_range'], prefix='price')


In [8]:
# Variables predictoras
X = df_encoded.drop(columns=['Estimated owners'])  # o el nombre de tu target

# Variable objetivo
y = df_encoded['Estimated owners']


In [10]:
# Verificá qué columnas no son numéricas
print(X.dtypes[X.dtypes == 'object'])


Name              object
About the game    object
Reviews           object
Header image      object
Website           object
Support url       object
Support email     object
Metacritic url    object
Notes             object
Developers        object
Publishers        object
Screenshots       object
Movies            object
dtype: object


In [11]:
# Eliminar columnas tipo texto que no fueron codificadas
columnas_objetables = X.select_dtypes(include='object').columns
X = X.drop(columns=columnas_objetables)


In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [13]:
# Train/Test Split
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

# División 80/20
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Escalado
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Entrenamiento
model = LogisticRegression(max_iter=1000, multi_class='multinomial', solver='lbfgs')  # para multiclase
model.fit(X_train_scaled, y_train)

# Predicciones
y_pred_iter1 = model.predict(X_test_scaled)


/Users/luisvillela/Desktop/UFM/2025-1/Data science/ProyectoDataScience/env/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


# Evaluación del modelo de Iteración 1

In [ ]:
from sklearn.metrics import classification_report, precision_score, recall_score, f1_score, confusion_matrix
from scipy.stats import ks_2samp

# Matriz de confusión
cm = confusion_matrix(y_test, y_pred_iter1)
print("Matriz de Confusión (Iteración 1):")
print(cm)

# Reporte general
print("\nReporte de Clasificación:")
print(classification_report(y_test, y_pred_iter1))

# Métricas macro y weighted
print("Precision (macro):", round(precision_score(y_test, y_pred_iter1, average='macro'), 3))
print("Recall (macro):", round(recall_score(y_test, y_pred_iter1, average='macro'), 3))
print("F1 Score (macro):", round(f1_score(y_test, y_pred_iter1, average='macro'), 3))

print("Precision (weighted):", round(precision_score(y_test, y_pred_iter1, average='weighted'), 3))
print("Recall (weighted):", round(recall_score(y_test, y_pred_iter1, average='weighted'), 3))
print("F1 Score (weighted):", round(f1_score(y_test, y_pred_iter1, average='weighted'), 3))


# 1. Crear versión binaria del target para KS
threshold = 100000
y_test_bin = (y_test >= threshold).astype(int)

# 2. Obtener índice de la clase positiva en predict_proba
clase_positiva = model.classes_[np.argmax(model.classes_ >= threshold)]
indice_clase = list(model.classes_).index(clase_positiva)

# 3. Probabilidades predichas para esa clase
y_prob = model.predict_proba(X_test_scaled)[:, indice_clase]

# 4. Separar según valor real
probs_pos = y_prob[y_test_bin == 1]
probs_neg = y_prob[y_test_bin == 0]

# 5. KS Score
ks_score = ks_2samp(probs_pos, probs_neg).statistic
print("KS Score:", round(ks_score, 3))


Matriz de Confusión (Iteración 1):
[[ 3398  1035     1     0     0     0     0     0     0     0     0     0
      0]
 [  206 13568   179    33    12     6     3     3     3     2     1     0
      0]
 [    1  1339   245    58    24    21     3     1     0     1     0     0
      0]
 [    0   495   149    59    51    28     8     0     1     2     0     0
      0]
 [    0   219   105    68    80    62    12     4     0     1     0     1
      0]
 [    0    76    45    37    74   151    23    13     1     2     0     0
      0]
 [    0     9     8     4    11    75    47    15     8     4     0     0
      0]
 [    0     7     1     1     4    25    25    31    16     5     2     0
      0]
 [    0     0     0     0     0     3     4    25    19     7     1     1
      0]
 [    0     0     0     0     0     1     1     1     8     2     1     0
      0]
 [    0     0     0     0     1     0     0     1     2     1     0     0
      1]
 [    0     0     0     0     0     0     0     0   

/Users/luisvillela/Desktop/UFM/2025-1/Data science/ProyectoDataScience/env/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/luisvillela/Desktop/UFM/2025-1/Data science/ProyectoDataScience/env/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/luisvillela/Desktop/UFM/2025-1/Data science/ProyectoDataScience/env/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

# 1. Convertimos X_train escalado a DataFrame
X_train_df = pd.DataFrame(X_train_scaled, columns=X.columns)

# 2. Agregamos constante
X_train_df_const = sm.add_constant(X_train_df)

# 3. Alineamos índices
X_train_df_const = X_train_df_const.reset_index(drop=True)
y_binary = (y_train == 0).astype(int).reset_index(drop=True)

# 4. Eliminar columnas con colinealidad (basado en condición del número)
def remove_collinear(X, threshold=30):
    from numpy.linalg import cond
    while cond(X) > threshold:
        corr_matrix = np.corrcoef(X.T)
        np.fill_diagonal(corr_matrix, 0)
        max_corr = np.unravel_index(np.argmax(np.abs(corr_matrix)), corr_matrix.shape)
        X = np.delete(X, max_corr[1], axis=1)
    return X

X_array_clean = remove_collinear(X_train_df_const.values)

# 5. Volvemos a convertir a DataFrame
X_clean_df = pd.DataFrame(X_array_clean)

# 6. Ajustamos el modelo
logit_model = sm.Logit(y_binary, X_clean_df)
result = logit_model.fit()

# 7. Mostramos resumen
print(result.summary())


/Users/luisvillela/Desktop/UFM/2025-1/Data science/ProyectoDataScience/env/lib/python3.13/site-packages/numpy/lib/_function_base_impl.py:3045: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/Users/luisvillela/Desktop/UFM/2025-1/Data science/ProyectoDataScience/env/lib/python3.13/site-packages/numpy/lib/_function_base_impl.py:3046: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


Optimization terminated successfully.
         Current function value: 0.346059
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:       Estimated owners   No. Observations:                89161
Model:                          Logit   Df Residuals:                    89157
Method:                           MLE   Df Model:                            3
Date:                Fri, 16 May 2025   Pseudo R-squ.:                  0.3046
Time:                        12:27:43   Log-Likelihood:                -30855.
converged:                       True   LL-Null:                       -44372.
Covariance Type:            nonrobust   LLR p-value:                     0.000
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
0             -1.8476      0.012   -155.782      0.000      -1.871      -1.824
1             -0.4273      0.

In [23]:
from sklearn.metrics import accuracy_score

# Accuracy general del modelo
accuracy = accuracy_score(y_test, y_pred_iter1)
print("Accuracy:", round(accuracy, 3))


Accuracy: 0.79
